# Common base Colpitts calculations
based on [Common Base Oscillator Design](./doc/Common-Base-Oscillator-Design-07june2023-1.pdf) and on [Chapter 6. Oscillator](./doc/18253023.pdf).  Remark that C1 and C2 are swapped in the two documents.

<figure>
    <img src="./doc/CB-colpitts_oscillator.png" alt="Colpitts-Oscillator-CB" style="width: 500px;"/>
    <figcaption>Colpitts Oscillator Common Base</figcaption>
</figure>

In [25]:
import math
# Given values
f0 = 100e6

# Calculate ω0
ω0 = 2*math.pi*f0
# Set Ic in the range 1 to 2mA (typical value)
Ic = 2e-3
# Calulate gm
gm = Ic/25e-3
# Calculate r_pi
r_pi = 1/gm
print(f"gm = {gm:.2f} S")
print(f"r_pi = {r_pi:.2f} Ω")
# Calculate impedance of C2 : X_C2 must be much smaller than r_pi (50% is ok), otherwise the formula is not valid.
X_C2 = r_pi/2
# Calculate C2
C2 = 1/(ω0*X_C2)
print(f"C2 = {C2*1e12:.2f} pF")

# Choose a standard value for C2
C2 = 47e-12
# Choose a standard value for C1 : C2/3 < C1 < C2
C1 = 47e-12
# Loop voltage gain
Avβ = 1 + C2/C1
print(f"Loop voltage gain Avβ = {Avβ:.2f}")
# Calculate equivalent capacitance C
C = (C1*C2)/(C1+C2)
# Calculate L
L = 1/(ω0**2 * C)
print(f"L = {L*1e9:.2f} nH")
# Negative resistance Rs
Rs = -gm / (ω0**2 * C1 * C2)
print(f"Negative resistance Rs = {Rs:.2f} Ω") # absolute value must be larger than Rs of the load to have oscillation

gm = 0.08 S
r_pi = 12.50 Ω
C2 = 254.65 pF
Loop voltage gain Avβ = 2.00
L = 107.79 nH
Negative resistance Rs = -91.73 Ω


# Parameters
1. For the varactar tuning range, we aim at a 20pF.  Because two varactors are in anti-series, the total capacitance range will only be 10pF.
2. Below we calculate the inductance for that tuning range.
    * 150nH yields 10pF ΔC, with Ceq varying from 17pF to 28pF.  This can work with C1=C2=15pF, impedance of C2 is no longer insignificant compared to the emitter resistor.
    * 100nH yields 16pF ΔC, with Ceq varying from 27pF to 42pF.
    * 82nH yields 20pF ΔC, with Ceq varying from 33pF to 52pF.  
3. Let's choose L = 82nH.
4. At least 10pF is needed for the varactors, so we choose C1||C2 = 22pF.  So C1 and C2 can be 39pF.

In [ ]:
import math
L = 100e-9 # Choose a lower value than the calculated one (100nH) because the varactor capacitance will add at least 10pF to C.
C = 39e-12/2  # Equivalent capacitance of C1 and C2 in series
# Calculate resonant frequency
f0 = 1 / (2*math.pi*math.sqrt(L*C))
print(f"Resonant frequency f0 = {f0/1e6:.2f} MHz")

# Calculate equivalent capacitance C
f0 = 107.9e6 # from simulation
C = 1 / ((2*math.pi*f0)**2 * L)
print(f"Equivalent capacitance Ceq = {C*1e12:.2f} pF")

# Calculate C_low for low frequency limit
f_if = 10.7e6
f_low = 88e6 - f_if
C_low = 1 / ((2*math.pi*f_low)**2 * L)
print(f"C_low = {C_low*1e12:.2f} pF")
# Calculate C_high for high frequency limit
f_high = 108e6 - f_if
C_high = 1 / ((2*math.pi*f_high)**2 * L)
print(f"C_high = {C_high*1e12:.2f} pF")
if C_high < C + 10e-12:
    print("Warning: C_high is less than Ceq + 10pF.  Either use smaller C1 and C2 or decrease L.")


# Calculate tuning capacitance range
ΔC = C_low - C_high
print(f"ΔC = {ΔC*1e12:.2f} pF")


Resonant frequency f0 = 113.97 MHz
Equivalent capacitance Ceq = 21.76 pF
C_low = 42.39 pF
C_high = 26.76 pF
ΔC = 15.64 pF


# References

## Common Base Colpitts and Clapp Oscillators
<figure>
  <img src="./doc/circuit.png" alt="Clapp Oscillator Common Base" width="400"/>
  <figcaption>Clapp Oscillator Common Base</figcaption>
</figure>

* [184N. Oscillator general topology, large signal Gm, amplitude and frequency calculation](https://www.youtube.com/watch?v=L7uazLiHC7o)
  * general derivation of oscillator frequency and amplitude
* [LC Oscillators | Intro to Analog Design | Harvey Mudd College | Video 25.2](https://www.youtube.com/watch?v=QcbPoFww6cg)
  * monotonous voice
  * peculiar way of doing the small signal analysis
* [MegaWattKS Semester Project - Radio Design 401, Episode 7](https://youtu.be/-rBG8U4wsOk?t=1972): not much info, no component values
  * Common base oscillator for 100MHz, built with an 2N5179 transistor.
  * [common base Colpitts oscillator](./doc/Common-Base-Oscillator-Design-07june2023-1.pdf), of which the original dirty version is [Oscillator design](./doc/010_Oscillators_and_VCOs.pdf).
  * Simulation works with 2mA collector current, Vcc=5V.  Quite distorted output waveform.
* [Oscillator Circuits: Types, Analysis, and Performance Factors](./doc/18253023.pdf)
  * colpitts and clapp
* [Activity: The Clapp Oscillator](https://wiki.analog.com/university/courses/electronics/comms-lab-clapp-osc)
  * includes design example with component values, but doesn't show where they come from
* R.Quan : Build your own transistor radios 
  * common base configuration
  * not too much detail

## Common Gate Colpitts
* [Double Conversion Pulse Counting FM Superhet Receiver With 10.7 MHz First IF Stage](https://www.philsvalveradiosite.co.uk/doubleconversionpulsecountingfmtuner_1.htm) : uses 2N3819 JFET

# Common Collector Colpitts and Clapp Oscillators
A common collector Colpitts oscillator is by default a Clapp oscillator, because the inductor is in series with an additional capacitor to ground. When C3 is much larger than C1 and C2, it has little effect on the oscillation frequency and is only there to protect the transistor biasing.  If C3 is much smaller than C1 and C2, the oscillating frequency depends mainly on C3 and the inductor.



In [3]:
## Common-collector Clapp oscillator calculations
import math
# Select Ic in the range 1 to 10mA (typical value)
Ic = 2.8e-3
# Select f0
f0 = 100e6
# Set Re (series resistance of the inductor + any additional resistance)
Re = 1
# Calculate ω0
ω0 = 2*math.pi*f0
# Calculate gm
gm = Ic/25e-3
# Select C1 at least 10x Cbe (=Cπ) to minimize its effect on frequency stability.  This value is not given, we can assume it to be tens of pF.
# Don't select C1 too high otherwise X_C2 will be too small and Av will be too low.
C1 = 100e-12
# Calculate X_C1
X_C1 = 1/(ω0*C1)
# Select C2 = C1
C2 = C1
# Gain of feedback network β
β = 1 + C2/C1
# Calculate X_C2
X_C2 = 1/(ω0*C2)
# Calculate Z_l
Z_l = X_C2**2 / Re
# Calulate Av
Av = gm * Z_l / (1 + gm * Z_l)
print(f"Voltage gain Av = {Av:.2f}.  Check that Av is close to 1.")
# Calculate loop gain Av*β
Avβ = Av * β
print(f"Loop voltage gain Avβ = {Avβ:.2f}.  Check that Avβ >> 1 so that the oscillator can start reliably.")
# Check gain margin another way
if Re < gm * X_C1 * X_C2:
    print(f"Gain margin check passed: Re < gm * X_C1 * X_C2: {Re:.2f} < {gm * X_C1 * X_C2:.2f}")
else:
    print(f"Gain margin check failed: Re >= gm * X_C1 * X_C2: {Re:.2f} >= {gm * X_C1 * X_C2:.2f}")

Voltage gain Av = 0.97.  Check that Av is close to 1.
Loop voltage gain Avβ = 1.93.  Check that Avβ >> 1 so that the oscillator can start reliably.
Gain margin check passed: Re < gm * X_C1 * X_C2: 1.00 < 28.37



## References
* [#96: Analysis & Design of a Typical Colpitts Oscillator](https://www.youtube.com/watch?v=TSKq5l7uuz4):
  * very detailed analysis, aligns very well with the devTtyS0 video
  * includes design example with component values
  * Clapp oscillator : C3 in series with the inductor to protect the bias of the transistor.  C3 is made much larger than C1 and C2, so that it has little effect on the oscillation frequency.
* [devTtyS0](https://www.youtube.com/watch?v=o-TFeAUumW8)
  * detailed analysis
    * A = gm * Zl / (1 + gm * Zl) (voltage gain of common collector stage), with Zl = Xc_2² / Re (with Re the ESR of the crystal, when not using an inductor) (only valid at the oscillation frequency)
    * β (gain of feedback network) = 1 + C2/C1 = vbe/ve -> so the gain only depends on the capacitors ratio (approximately)
    * A * β > 1 converts to : gm * X_C1 * X_C2 > Re (Re is the ESR of the crystal, or series resistor if an inductor is used)
  * Colpitts oscillator typical have distorted outputs because of the non-linearity of the transistor
  * includes design example with component values
  * Transistor biasing : 
    * emitter voltage at VCC/2 (minimizes temp effect on vbe as a large emitter voltage is in series with vbe)
    * emitter current at 1mA (low power dissipation, but high hfe)
    * base resistors R1 and R2 : (R1||R2) << 10 * R_E (emitter resistor) to keep base voltage stable
  * Load capacitance effect
    * C1 = 10x Cbe of transistor
    * C2 = C1 -> loop gain is 2
  * Stray parasitics
    * Re (use ESR of crystal, when not using an inductor)
  * Power output
  * gain margin, Loop gain
     Re < Xc1 * Xc2 * gm
  * Loaded Q factor : impedance seen from the crystal or inductor terminals
    * Capacitive loading : C1 and C2 in series -> must be equal to defined capacitive loading of the crystal.
    * Resistive loading : R1 || R2 -> must be much smaller than ESR of crystal
    * Convert series model to parallel model
* [Oscillators for FM radio](https://www.edaboard.com/threads/oscillators-for-fm-radio.413247/), [offline version](./doc/[SOLVED]%20-%20Oscillators%20for%20FM%20radio%20_%20Forum%20for%20Electronics.pdf) : simulation OK
* [Lund University : Local Oscillator for FM broadcast band 88-108 MHz](./doc/002401269.pdf) : simulation OK
* [Zetex Applications Note 9](./doc/an9.pdf)
  * fig. 9 : common collector clapp oscillator on page 8.
* [Clapp Oscillator Theory - David S. Ricketts](./doc/Clapp%20Oscillator%20Theory%20-%20David%20S.%20Ricketts.pdf)

<figure>
  <img src="./doc/Clapp_88MHz-108MHz_from_EDABoard.jpg" alt="Clapp oscillator" width="500"/>
  <figcaption>Clapp oscillator : simulation of the oscillator works (buffer not simulated)</figcaption>
</figure>
<figure>
  <img src="./doc/OSFM963.jpg" alt="Clapp oscillator" width="500"/>
  <figcaption>Clapp Oscillator Using BJT</figcaption>
</figure>

## Common drain Colpitts
* [Clapp Oscillators](https://www.electronics-tutorials.com/oscillators/colpitts-oscillators.htm)
* [Clapp Oscillator EDN](./doc/Clapp%20oscillator%20-%20EDN.pdf)
* [Reproducible Low Noise Oscillators](./doc/Reproducible%20Low%20Noise%20Oscillators.pdf)

# Common Emitter Colpitts Oscillators
* [Colpitts Oscillator Report](https://github.com/AniruthSuresh/Colpitts-Oscillator/blob/main/Report.pdf)
  * includes design example with component values
* [Colpitts Oscillator Circuit Analysis (7 - Oscillators)](https://youtu.be/ES-kcNR4Ln0)
  * includes analysis and design example with component values
* [Understanding VCO Concepts](https://blog.minicircuits.com/understanding-vco-concepts/) & [AN95-007](https://www.minicircuits.com/appdoc/AN95-007.html)
  * common collector Colpitts oscillator : load on collector does not affect oscillation frequency
  * common base Colpitts oscillator : better for high frequency operation
* [Clapp Oscillator: Frequency Formula And Circuit Diagram | Electrical4U](./doc/Clapp%20Oscillator_%20Frequency%20Formula%20And%20Circuit%20Diagram%20_%20Electrical4U.pdf) : explains advantages of Clapp over Colpitts



* [HF/VHF/UHF TEST OSCILLATOR](http://www.cappels.org/dproj/bandswitchtestosc/bs_test_oscillator.html)
  * uses JFETs
* Hayward IRFD
  * "Figure 7.8 The Clapp variation of the Colpitts oscillator"
  * Deep-dive mathematical analysis with Y-parameters
  * Analysis is done using common base configuration
  * Common collector configuration : practical and experimentally interesting : 
    * series resistor might be added between collector and Vcc : provides output that is well isolated from the rest of the circuit

* Less interesting: 
  * Jessop, p.85 : oscillator design
  * [Clapp oscillator explained](https://everything.explained.today/Clapp_oscillator/)
  * [Clapp oscillator - TIDJMA](https://www.tidjma.tn/fr/electrical/clapp-oscillator-/)
  * [The Clapp Oscillator - and How! February 1953 QST](https://www.rfcafe.com/references/qst/clapp-oscillator-february-1953-qst.htm)

# Tuning

In [4]:
# Finding capacitance looking into R1 and R2 (from CC_Clapp_design1.asc)
import math
L = 330e-9

# LTSpice Simulation values
# -------------------------
f, C5 = 63.2e6, 50e-12 # C_other = 19.2pF
#f, C5 = 55.0e6, 100e-12 # C_other = 25.4pF
#f, C5 = 52.9e6, 150e-12 # C_other = 27.4pF

Ceq = 1 / ( (2*math.pi*f)**2 * L )
C_osc = 1 / ( 1/Ceq - 1/C5 )
print(f"C_osc (capacitance looking into R1 and R2) = {C_osc*1e12:.2f} pF")
C_other = 1 / ( 1/C5 + 1/C_osc )
print(f"C_other (capacitance looking into C5 from left to right)= {C_other*1e12:.2f} pF")

C_osc (capacitance looking into R1 and R2) = 31.21 pF
C_other (capacitance looking into C5 from left to right)= 19.22 pF


In [5]:
# Find capacitance C for given f0 and L
import math
L = 390e-9

f1 = 88e6
f2 = 108e6
iff = 10.7e6
f_l = f1 - iff
f_h = f2 - iff
C_l = 1 / ( (2*math.pi*f_l)**2 * L )
C_h = 1 / ( (2*math.pi*f_h)**2 * L )
print(f"C_l = {C_l*1e12:.2f} pF at {f_l/1e6:.2f} MHz")
print(f"C_h = {C_h*1e12:.2f} pF at {f_h/1e6:.2f} MHz")

C_varactor_l = 1 / ( 1 / C_l - 1 / C_other)
print(f"C_varactor_l = {C_varactor_l*1e12:.2f} pF")
C_varactor_h = 1 / ( 1 / C_h - 1 / C_other)
print(f"C_varactor_h = {C_varactor_h*1e12:.2f} pF")

C_l = 10.87 pF at 77.30 MHz
C_h = 6.86 pF at 97.30 MHz
C_varactor_l = 25.02 pF
C_varactor_h = 10.67 pF


Simulation shows that the equivalent capacitance of the varactor diode should be 9pF to 25pF to cover the 88MHz to 108MHz band, when using L=390nH and C5=100pF.  So that's a range of 16pF.  As we will be using two diodes in anti-series, each diode must cover a capacitance range of 18pF to 50pF.  We want to keep the voltage across each diode as high as possible to avoid driving them into forward conduction.  So we will use a tuning range of 8pF (5V) to 40pF (1.5V).  That means we will need about 5pF of fixed capacitance in parallel with the diodes to reach the required capacitance range.

For the oscillation amplitude, we have two contradicting requirements.  On the one hand, we want a high amplitude to drive the mixer stage properly.  On the other hand, we want a low amplitude to avoid driving the varactor diodes into forward conduction.  We will aim for an oscillation amplitude of about 1V peak-to-peak, which should be a good compromise.

# Buffer amplifier
At these high frequencies, each buffer amplifier stage has a current gain of only 6.  So the resistance seen looking into the base is only 6 times the resistance seen looking into the emitter.  Therefore, we need at least two buffer stages to isolate the oscillator from the mixer stage.

1. Design it from back to front.
2. Replace biasing resistors by thevenin equivalent and replace emitter resistor by a current source.
3. Set signal source to 700mV amplitude.  With a 1x voltage gain, it will result in 7dBm into 50 ohms, which is 15mArms
4. Tune the current source to get lowest distortion at about 7dBm output power.  It will be in the order of 15mA collector current.
5. Either replace the current source by an actual current source circuit, or replace it by an emitter resistor.
6. Tune the thevenin equivalent resistance and voltage to get the lowest distortion at that output power.  Ideally, the thevenin equivalent resistance should be as high as possible to avoid loading the preceding stage.
7. Then replace the thevenin equivalent by the preceding CC-amplifier stage.
8. Again have the emitter resistor replaced by a current source, and tune the current source to get lowest distortion at that output power.
9. Again replace the biasing resistors by thevenin equivalent.
10. Finally, replace the thevenin equivalent by the resistors.